# Test Article

Загружаем библиотеки

In [1]:
from transformers import AutoTokenizer, AutoModel, pipeline, logging

In [2]:
modelpath = "../model/fine-train"
# modelpath = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained(modelpath)

Some weights of BertModel were not initialized from the model checkpoint at ../model/fine-train and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
text = '''Ранее в четверг лидеры Европейского союз единогласно согласовали выделение Украине макрофинансовой помощи в размере 50 миллиардов евро на период до 2027 года с условием ежегодной отчетности со стороны ЕК и дебатов в Евросовете. Как сообщил премьер-министр Польши Дональд Туск, главу венгерского правительства Виктора Орбана "удалось убедить".
Позиция Будапешта оставалась главным препятствием для выделения денег киевским властям. Как выяснила газета Financial Times, лидеры стран Евросоюза могли публично объявить на саммите о неконструктивном поведении Орбана и отказать Венгрии в разморозке средств из европейских фондов.
Как подчеркивал сам венгерский премьер, такое развитие событий привело бы к армагеддону. Он заявил, что европейцам 50 миллиардов евро нужны не меньше, чем украинцам, но предложил компромисс: не выделять всю сумму сразу на четыре года, а утверждать финансирование для Киева ежегодно и единогласно.'''

In [4]:
import re

tr = 0.0001
logging.set_verbosity_error()

def mask_bert_sent(text, model, tokenizer):
    unmasker = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    p = re.compile(r'[\w-]+')
    iter = p.finditer(text)
    listErr = []
    for match in iter:
        if match.start() == 0:
            masktext = "[MASK]" + text[match.end():]
        elif match.end() == len(text):
            masktext = text[:match.start()] + "[MASK]"
        else:
            masktext = text[:match.start()] + "[MASK]" + text[match.end():]
        
        res = unmasker(masktext, targets=[match.group()])
        if res[0]['score'] < tr:
            errorrDesc = {
                "word": match.group(),
                "start": match.start(),
                "end": match.end(),
                "prob": res[0]['score']
            }
            listErr.append(errorrDesc)
    return listErr


In [5]:
errors = mask_bert_sent(text, modelpath, tokenizer)
if len(errors) > 0:
    for e in errors:
        print (e)

{'word': 'союз', 'start': 36, 'end': 40, 'prob': 4.973573140887311e-06}
{'word': 'макрофинансовой', 'start': 83, 'end': 98, 'prob': 2.9836128305760212e-05}
{'word': 'дебатов', 'start': 206, 'end': 213, 'prob': 6.682212551822886e-05}
{'word': 'Евросовете', 'start': 216, 'end': 226, 'prob': 3.2444997486891225e-05}
{'word': 'неконструктивном', 'start': 528, 'end': 544, 'prob': 5.306245384417707e-06}
{'word': 'Орбана', 'start': 555, 'end': 561, 'prob': 9.039770247909473e-07}
{'word': 'разморозке', 'start': 583, 'end': 593, 'prob': 7.1426114800488e-08}
{'word': 'армагеддону', 'start': 701, 'end': 712, 'prob': 4.440678491590688e-09}
{'word': 'европейцам', 'start': 729, 'end': 739, 'prob': 3.131354969809763e-05}
